# Movie Recommendation System GPU COPY FILE 2

In [11]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import time
from sklearn.svm import SVR
from sklearn.neighbors import NearestNeighbors
import helper_gpu as hp

# Reading Data

In [12]:
query=pd.read_csv("/home/avirup/Downloads/ml-100k/u1.base",header=None,delimiter="\t",names=["user id", "item id","rating","timestamp"])

In [13]:
query_test=pd.read_csv("/home/avirup/Downloads/ml-100k/u1.test",header=None,delimiter="\t",names=["user id", "item id","rating","timestamp"])

In [14]:
user=pd.read_csv("/home/avirup/Downloads/ml-100k/user_full.csv",header=None)

In [15]:
item_2=pd.read_csv("/home/avirup/Downloads/ml-100k/u.item",header=None,delimiter="|",encoding='ISO-8859-1')

In [16]:
item=pd.read_csv("/home/avirup/Downloads/ml-100k/final_item.csv",header=None)

In [17]:
user_2=pd.read_csv("/home/avirup/Downloads/ml-100k/user_state.csv",header=None)

# KNN

## Item Based 

In [18]:
# Generating the movies vs genre matrix
genre_movie_embedding=item.iloc[:,2:].values

In [19]:
# Filtering out those item that have already been rated
def item_based(user_id,query,query_test,item):
    #print(genre_movie_embedding)
    genre_movie_embedding_train=[]
    for y in query[query["user id"]==user_id]["item id"].values:
        genre_movie_embedding_train.append(genre_movie_embedding[y-1,:])
        
    neigh = NearestNeighbors(n_neighbors=1,metric="cosine")
    neigh.fit(genre_movie_embedding_train)

    
# Glink is a class of graph nodes and edge weights put in a tuple (,). The float comparison operators are polymorhed
#into this class therefore these data structures can be sorted by any sorting algorithm based on the second
#entry of the tuple (,....).

    
    distance=[]
    for i in range(len(item)):
        if item.iloc[i,0] not in query[query["user id"]==user_id]["item id"].values:
            distance.append(hp.glink((item.iloc[i,0],neigh.kneighbors([genre_movie_embedding[i,:]])[0][0,0])))
        
        
    distance=np.sort(distance)
    reco=[]
    for i in range(50):
        reco.append(distance[i].link[0])
        
    match=0
    for x in reco:
        if x in query_test[query_test["user id"]==user_id]["item id"].values:
            match=match+1
    return(match,len(query_test[query_test["user id"]==user_id]))

In [20]:
temp1=0
temp2=0
for x in user[0].values:
    print(x,end="\r")
    out=item_based(user_id=x,query=query,query_test=query_test,item=item)
    temp1=temp1+out[0]
    temp2=temp2+out[1]
   # print(out[0],out[1])
print(temp1/temp2)

0.0187


## User Based

In [ ]:
X=feature=hp.feature_matrix_construct(features=[user[1].unique(),user[2].unique(),user[3].unique(),user[5].unique()],columns_index=[1,2,3,5],df=user)

In [33]:
neigh = NearestNeighbors(n_neighbors=40,metric="cosine")

In [34]:
neigh.fit(X)

NearestNeighbors(metric='cosine', n_neighbors=40)

In [35]:
neigh.kneighbors([X[10,:]])

(array([[0.  , 0.  , 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.5 , 0.5 , 0.5 ,
         0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
         0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
         0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ]]),
 array([[ 10, 718,  17, 856, 759, 555, 166, 277, 416, 154, 701, 858, 150,
         420, 148, 423, 900, 164, 436, 708, 860, 449, 687, 169,  78, 651,
         504, 891, 493, 489, 884, 101, 668,   1, 459, 456, 119, 329, 128,
         332]]))

In [42]:
tot=0
deno_tot=0
for k in range(len(user)):
    print(k,end="\r")
    Y=neigh.kneighbors([X[k,:]])
    reco=[]
    for i in range(len(Y[0][0])):
        if Y[0][0][i]==0:
            reco.append(Y[1][0][i])
    #print(reco)

    for x in reco:
        if x !=k:
            check1=query["user id"]==x+1
            check2=query["rating"]==5
            tot=tot+(len(query[check1 & check2].merge(query_test[query_test["user id"]==k+1],\
                    how="inner", left_on="item id",right_on="item id")))
            deno_tot=deno_tot+len(query[check1 & check2])
print(tot/deno_tot)

0.04565217391304348
